# PyACP performance test with slower network

This notebook tests the PyACP performance when run with a slower network connection. It uses the Linux ``tc-netem`` command to emulate a slower (latency- and bandwidth-restricted) network in a Docker container.

---
**NOTE**

- not all Linux kernels support ``tc-netem``. For example, it cannot be run on WSL2.
- this notebook may become outdated as the PyACP codebase evolves. It is not regularly checked for compatibility, and may need to be updated.

## Build docker image

In [ ]:
!DOCKER_BUILDKIT=1 docker build --tag pyacp-benchmark-runner -f ../dockerfiles/ConnectionTest.Dockerfile .

## Launch servers

In [ ]:
import ansys.acp.core as pyacp

In [ ]:
slow_launch = pyacp.DockerComposeLaunchConfig(
    image_name_pyacp="pyacp-benchmark-runner",
    compose_file="../docker-compose/docker-compose-benchmark.yaml",
    environment_variables={"PYACP_DELAY": "200ms", "PYACP_RATE": "1mbit"},
)
normal_launch = pyacp.DockerComposeLaunchConfig()


In [ ]:
slow_acp = pyacp.launch_acp(config=slow_launch)

In [ ]:
normal_acp = pyacp.launch_acp(config=normal_launch)

## Run example

In [ ]:
def run_example(acp_instance):
    f = acp_instance.upload_file(local_path="../examples/data/class40/class40.cdb")
    m = acp_instance.import_model(path=f, format="ansys:cdb")
    mg = m.create_modeling_group()
    for _ in range(100):
        mg.create_modeling_ply()
    acp_instance.clear()

In [ ]:
run_example(normal_acp)

In [ ]:
run_example(slow_acp)